### Implementing Data Validation using Protobuf in a Distributed System
**Description**: Use gRPC to implement a distributed system that validates messages using
Protobuf.

**Steps**:
1. Create a .proto file for gRPC service.
2. Implement server-side validation
    - Create a gRPC server
    - Bind the server to an address
    - Start server

In [ ]:
# Write your code from here
# Step 1: .proto file (save as validate.proto)

"""
syntax = "proto3";

package validation;

service Validator {
  rpc ValidateData (DataMessage) returns (ValidationResult) {}
}

message DataMessage {
  string name = 1;
  int32 age = 2;
}

message ValidationResult {
  bool is_valid = 1;
  string message = 2;
}
"""

# Step 2: Generate Python code from proto file
# Run in terminal (requires grpcio-tools): 
# python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. validate.proto

# Step 3: Implement gRPC server

import grpc
from concurrent import futures
import time
import validate_pb2
import validate_pb2_grpc

class ValidatorServicer(validate_pb2_grpc.ValidatorServicer):
    def ValidateData(self, request, context):
        if not request.name:
            return validate_pb2.ValidationResult(is_valid=False, message="Name cannot be empty")
        if request.age < 0 or request.age > 120:
            return validate_pb2.ValidationResult(is_valid=False, message="Age is not valid")
        return validate_pb2.ValidationResult(is_valid=True, message="Validation successful")

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    validate_pb2_grpc.add_ValidatorServicer_to_server(ValidatorServicer(), server)
    server.add_insecure_port('[::]:50051')
    print("Server started on port 50051...")
    server.start()
    try:
        while True:
            time.sleep(86400)
    except KeyboardInterrupt:
        server.stop(0)

if __name__ == '__main__':
    serve()
